# ML testing: experiment #1- unharmonized

This notebook involves testing for the MRI conference abstract. This notebook shows unharmonized StrokeMRI and TOP based models, and how they perform on each other

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys
import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
import seaborn as sns

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config
import cvasl.harmony as har

### import data , put sex numerical

In [ ]:
filepath_mri = '../open_work/internal_results/cleaned_pvc2s/' 
filename_mri = os.path.join(filepath_mri,'StrokeMRI_pvc2c.csv') 
filepath_top = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top = os.path.join(filepath_top,'TOP_pvc2c.csv') 
TOP = pd.read_csv(filename_top)
StrokeMRI = pd.read_csv(filename_mri)
TOP = TOP.drop(TOP.columns[0],axis=1)
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)

# Now we need to flip the sex back to numbers for a correlation
sex_mapping = {'F':0,'M':1}
TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
TOP.head(3)

In [ ]:
StrokeMRI.head(3)

## choose whether to log columns

In [ ]:
# examine distributions visually
possibilities = StrokeMRI.columns.to_list()
def see_feature_on_two(a):
    sns.displot(StrokeMRI, x= a, hue= 'sex'), plt.title('StrokeMRI')
    sns.displot(TOP, x= a, hue= 'sex'), plt.title('TOP')
    return a
w = interactive(see_feature_on_two,
                a=widgets.Dropdown(
                    options= possibilities,
                    value='age',
                    description='feature',
                    disabled=False))
w

In [ ]:
data = StrokeMRI
features_to_log = widgets.SelectMultiple(
    options=data.columns.tolist(),
    value=[],
    description='Features',
    disabled=False
)
features_to_log

In [ ]:
list(features_to_log.value)

In [ ]:
StrokeMRI = har.log_out_columns(StrokeMRI, list(features_to_log.value))
TOP = har.log_out_columns(TOP, list(features_to_log.value))

## add binned column on age

In [ ]:
StrokeMRI = har.bin_dataset(StrokeMRI, 'age', num_bins=4, graph = True)
TOP =  har.bin_dataset(TOP, 'age', num_bins=4, graph = True)
TOP.columns

In [ ]:
StrokeMRI[['age','binned']].sort_values(by='age')
#loc[StrokeMRI['binned'] == 3]

In [ ]:
StrokeMRI.head(3)

In [ ]:
StrokeMRI.isna().sum()

In [ ]:
## need definitioin for stiff binning and choice

In [ ]:
# def static_bin_age(dataframe):
#     dataframe['static_bin_age'] = 0
#     dataframe.loc[dataframe['age'] > 40]['static_bin_age'] = 1
#     #dataframe['static_bin_age']= 0
#     return dataframe
    
    
# StrokeMRI = static_bin_age(StrokeMRI)
# TOP =  static_bin_age(TOP)
# TOP

## Build ML models based on StrokeMRI

In [ ]:
ml_matrix = StrokeMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
linr_k_frame, linr_y_frame, models = sep.stratified_one_category_shuffle_split('linear regression', 'unharm_mri_linr', LinearRegression(), ml_matrix, X, y, printed=True)

In [ ]:
linr_k_frame

In [ ]:
linr_k_frame.to_csv('unharmonized_mri_linr_k_frame.csv')

In [ ]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

In [ ]:
linr_y_frame

In [ ]:
linr_y_frame.to_csv('unharmonized_mri_linr_y_frame.csv')

In [ ]:
linr = models[0]
linr[0]

In [ ]:
#models

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_mri_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_mri_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_mri_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_mri_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_mri_linr4.sav'))

In [ ]:
# llreg_k_frame, llreg_y_frame, models = sep.frame_a_model_sex_split('lasso regression', 'unharm_mri_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X, y)
# llreg_k_frame

In [ ]:
llreg_k_frame, llreg_y_frame, models =  sep.stratified_one_category_shuffle_split('lasso regression', 'unharm_mri_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X, y)
llreg_k_frame

In [ ]:
llreg_k_frame.to_csv('unharmonized_mri_llreg_k_frame.csv')

In [ ]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

In [ ]:
llreg_y_frame

In [ ]:
llreg_y_frame.to_csv('unharmonized_mri_llreg_y_frame.csv')

In [ ]:
llreg = models[0]
llreg[0]

In [ ]:
## optional save models
#joblib.dump(llreg[0], ('../result_models/'+ 'unharm_mri_llreg0.sav'))
#joblib.dump(llreg[1], ('../result_models/'+ 'unharm_mri_llreg1.sav'))
#joblib.dump(llreg[2], ('../result_models/'+ 'unharm_mri_llreg2.sav'))
#joblib.dump(llreg[3], ('../result_models/'+ 'unharm_mri_llreg3.sav'))
#joblib.dump(llreg[4], ('../result_models/'+ 'unharm_mri_llreg4.sav'))

In [ ]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_one_category_shuffle_split('decision tree', 'unharm_mri_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X, y)
dtree_k_frame

In [ ]:
dtree_k_frame.to_csv('unharmonized_mri_dtree_k_frame.csv')

In [ ]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

In [ ]:
dtree_y_frame

In [ ]:
dtree_y_frame.to_csv('unharmonized_mri_dtree_y_frame.csv')

In [ ]:
dtree = models[0]
dtree[0]

In [ ]:
regr_k_frame, regr_y_frame, models = sep.stratified_one_category_shuffle_split('MLP regression', 'unharm_mri_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X, y)
regr_k_frame

In [ ]:
regr_k_frame.to_csv('unharmonized_mri_regr_k_frame.csv')

In [ ]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

In [ ]:
regr_y_frame

In [ ]:
regr_y_frame.to_csv('unharmonized_mri_regr_y_frame.csv')

In [ ]:
regr = models[0]
regr[0]

### Note I'm not actually saving the mlp or svr models here because they are not the best performers. if neccesary this can be added

In [ ]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_one_category_shuffle_split('support vector reg poly2', 'unharm_mri_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X, y)
svrp2_k_frame

In [ ]:
svrp2_k_frame.to_csv('unharmonized_mri_svrp2_k_frame.csv')

In [ ]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

In [ ]:
svrp2_y_frame

In [ ]:
svrp2_y_frame.to_csv('unharmonized_mrisvrp2_y_frame.csv')

In [ ]:
svrp2 = models[0]
svrp2[0]

In [ ]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_one_category_shuffle_split('elasticnetCV', 'unharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X, y)
eregr_k_frame

In [ ]:
eregr_k_frame.to_csv('unharmonized_eregr_k_frame.csv')

In [ ]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

In [ ]:
eregr_y_frame

In [ ]:
eregr_y_frame.to_csv('unharmonized_mri_eregr_y_frame.csv')

In [ ]:
eregr = models[0]
eregr[0]

In [ ]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_one_category_shuffle_split('extra trees', 'unharm_mri_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X, y)
etreg_k_frame

In [ ]:
etreg_k_frame.to_csv('unharomized_etreg_k_frame.csv')

In [ ]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

In [ ]:
etreg_y_frame

In [ ]:
etreg_y_frame.to_csv('unharmonized_etreg_y_frame.csv')

In [ ]:
etreg = models[0]
etreg[0]

In [ ]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'unharm_mri_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'unharm_mri_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'unharm_mri_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'unharm_mri_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'unharm_mri_etreg4.sav'))

Show results ON AVERAGE for each model

In [ ]:
mri_based_unharmonized_on_testmri =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mri_based_unharmonized_on_testmri

In [ ]:
mri_based_unharmonized_on_testmri.to_csv('mri_based_unharmonized_on_testmri_AVERAGES.csv')

## Now we will build  models based on the whole unharmonized StrokeMRI dataset, and apply them to TOP. 

In [ ]:
ml_matrix = StrokeMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [ ]:
MRIlinr = LinearRegression()
MRIlinr.fit(X_train, y_train)

In [ ]:
MRIllreg = linear_model.LassoLars(alpha=0.01)
MRIllreg.fit(X_train, y_train)

In [ ]:
MRIeregr = ElasticNetCV(cv=5, random_state=17)
MRIeregr.fit(X_train, y_train)


In [ ]:
MRIetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MRIetreg.fit(X_train, y_train)

In [ ]:
##  Save these four best models

In [ ]:

## optional save models
#joblib.dump(MRIlinr, ('../result_models/' + 'unharm_whole_mri_linr.sav'))
#joblib.dump(MRIllreg, ('../result_models/'+ 'unharm_whole_mri_llreg1.sav'))
#joblib.dump(MRIeregr, ('../result_models/'+ 'unharm_whole_mri_eregr3.sav'))
#joblib.dump(MRIetreg, ('../result_models/'+ 'unharm_whole_mri_etreg4.sav'))

# Running whole MRI model over TOP dataset

In [ ]:
top_ml_matrix = TOP.drop('participant_id', axis=1)
X_top = top_ml_matrix.drop('age', axis =1)
X_top = X_top.values
X_top = X_top.astype('float')
y_top = top_ml_matrix['age'].values
y_top=y_top.astype('float')

In [ ]:
X_top_test = X_top
y_top_test = y_top


In [ ]:
y_top_pred = MRIlinr.predict(X_top_test)

In [ ]:
# print('R2 score Linear regression: %.3f' % linr.score(X_top_test,y_top_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
# print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'linear regression',
    'unharm_whole_mri_linr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    MRIlinr.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
linr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#linr_results

In [ ]:
linr_y_test = y_top_test
linr_y_pred = y_top_pred
linr_compare = pd.DataFrame(
    {'y_test_real_age': linr_y_test,
     'linr_y_pred_age': linr_y_pred,
    })
linr_compare = linr_compare.reset_index()
linr_compare

In [ ]:
linr_compare.to_csv('whole_mri_linr_compare_on_top.csv')

In [ ]:
y_top_pred = MRIllreg.predict(X_top_test)

In [ ]:
data= [[
    'lasso regression',
    'unharm_whole_mri_llreg.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    MRIllreg.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
llreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results

In [ ]:
llreg_y_test = y_top_test
llreg_y_pred = y_top_pred
llreg_compare = pd.DataFrame(
    {'y_test_real_age': llreg_y_test,
     'linr_y_pred_age': llreg_y_pred,
    })
#llreg_compare = llreg_compare.reset_index()
llreg_compare

In [ ]:
llreg_compare.to_csv('whole_mri_llreg_compare_on_top.csv')

In [ ]:
y_top_pred = MRIeregr.predict(X_top_test)

In [ ]:
data= [[
    'elasticnetCV',
    'unharm_whole_mri_linr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    MRIeregr.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
eregr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#eregr_results

In [ ]:
eregr_y_test = y_top_test
eregr_y_pred = y_top_pred
eregr_compare = pd.DataFrame(
    {'y_test_real_age': eregr_y_test,
     'linr_y_pred_age': eregr_y_pred,
    })
#eregr_compare = eregr_compare.reset_index()
eregr_compare

In [ ]:
eregr_compare.to_csv('whole_mri_eregr_compare_on_top.csv')

In [ ]:
y_top_pred = MRIetreg.predict(X_top_test)

In [ ]:
data= [[
    'extra trees',
    'unharm_mri_linr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    MRIetreg.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
etreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#etreg_results

In [ ]:
etreg_y_test = y_top_test
etreg_y_pred = y_top_pred
etreg_compare = pd.DataFrame(
    {'y_test_real_age': etreg_y_test,
     'linr_y_pred_age': etreg_y_pred,
    })
#etreg_compare = etreg_compare.reset_index()
etreg_compare

In [ ]:
etreg_compare.to_csv('whole_mri_etreg_compare_on_top.csv')

In [ ]:
mri_based_unharmonized_on_top =pd.concat([linr_results,
                   llreg_results,
                   eregr_results,
                  etreg_results],
                  axis=0)
mri_based_unharmonized_on_top

## Now we will run the exact opposite process.
1. We will explore TOP based models via k-folded results, 
2. We will make a general unahrmonized TOP model (based off all TOP)
3. We will apply the best of these model to the StrokeMRI dataset

### Build ML models based on TOP

In [ ]:
ml_matrix = TOP.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
linr_k_frame, linr_y_frame, models = sep.stratified_one_category_shuffle_split('linear regression', 'unharm_top_linr', LinearRegression(), ml_matrix, X, y)

In [ ]:
linr_k_frame

In [ ]:
linr_k_frame.to_csv('unharmonized_top_linr_k_frame.csv')

In [ ]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

In [ ]:
linr_y_frame

In [ ]:
linr_y_frame.to_csv('unharmonized_top_linr_y_frame.csv')

In [ ]:
linr = models[0]
linr[0]

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'unharm_top_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'unharm_top_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'unharm_top_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'unharm_top_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'unharm_top_linr4.sav'))

In [ ]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_one_category_shuffle_split('lasso regression', 'unharm_top_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X, y)
llreg_k_frame

In [ ]:
llreg_k_frame.to_csv('unharmonized_top_llreg_k_frame.csv')

In [ ]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

In [ ]:
llreg_y_frame

In [ ]:
llreg_y_frame.to_csv('unharmonized_top_llreg_y_frame.csv')

In [ ]:
llreg = models[0]
llreg[0]

In [ ]:
## optional save models
#joblib.dump(llreg[0], ('../result_models/'+ 'unharm_top_llreg0.sav'))
#joblib.dump(llreg[1], ('../result_models/'+ 'unharm_top_llreg1.sav'))
#joblib.dump(llreg[2], ('../result_models/'+ 'unharm_top_llreg2.sav'))
#joblib.dump(llreg[3], ('../result_models/'+ 'unharm_top_llreg3.sav'))
#joblib.dump(llreg[4], ('../result_models/'+ 'unharm_top_llreg4.sav'))

In [ ]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_one_category_shuffle_split('decision tree', 'unharm_top_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X, y)
dtree_k_frame

In [ ]:
dtree_k_frame.to_csv('unharmonized_top_dtree_k_frame.csv')

In [ ]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

In [ ]:
dtree_y_frame

In [ ]:
dtree_y_frame.to_csv('unharmonized_top_dtree_y_frame.csv')

In [ ]:
dtree = models[0]
dtree[0]

In [ ]:
regr_k_frame, regr_y_frame, models = sep.stratified_one_category_shuffle_split('MLP regression', 'unharm_top_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X, y)
regr_k_frame

In [ ]:
regr_k_frame.to_csv('unharmonized_top_regr_k_frame.csv')

In [ ]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

In [ ]:
regr_y_frame

In [ ]:
regr_y_frame.to_csv('unharmonized_top_regr_y_frame.csv')

In [ ]:
regr = models[0]
regr[0]

### Note I'm not actually saving the mlp or svr models here because they are not the best performers. if neccesary this can be added

In [ ]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_one_category_shuffle_split('support vector reg poly2', 'unharm_top_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X, y)
svrp2_k_frame

In [ ]:
svrp2_k_frame.to_csv('unharmonized_top_svrp2_k_frame.csv')

In [ ]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

In [ ]:
svrp2_y_frame

In [ ]:
svrp2_y_frame.to_csv('unharmonized_top_svrp2_y_frame.csv')

In [ ]:
svrp2 = models[0]
svrp2[0]

In [ ]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_one_category_shuffle_split('elasticnetCV', 'unharm_top_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X, y)
eregr_k_frame

In [ ]:
eregr_k_frame.to_csv('unharmonized_top_eregr_k_frame.csv')

In [ ]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

In [ ]:
eregr_y_frame

In [ ]:
eregr_y_frame.to_csv('unharmonized_top_eregr_y_frame.csv')

In [ ]:
eregr = models[0]
eregr[0]

In [ ]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_one_category_shuffle_split('extra trees', 'unharm_top_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X, y)
etreg_k_frame

In [ ]:
etreg_k_frame.to_csv('unharomized_top_etreg_k_frame.csv')

In [ ]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

In [ ]:
etreg_y_frame

In [ ]:
etreg_y_frame.to_csv('unharmonized_top_etreg_y_frame.csv')

In [ ]:
etreg = models[0]
etreg[0]

In [ ]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'unharm_top_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'unharm_top_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'unharm_top_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'unharm_top_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'unharm_top_etreg4.sav'))

Show results ON AVERAGE for each model

In [ ]:
top_based_unharmonized_on_testtop =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
top_based_unharmonized_on_testtop

In [ ]:
top_based_unharmonized_on_testtop.to_csv('top_based_unharmonized_on_testtop_AVERAGES.csv')

## Now we will build  models based on the whole unharmonized StrokeTOP dataset, and apply them to StrokeMRI. 

In [ ]:
ml_matrix = TOP.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')

In [ ]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [ ]:
TOPlinr = LinearRegression()
TOPlinr.fit(X_train, y_train)

In [ ]:
TOPllreg = linear_model.LassoLars(alpha=0.01)
TOPllreg.fit(X_train, y_train)

In [ ]:
TOPeregr = ElasticNetCV(cv=5, random_state=17)
TOPeregr.fit(X_train, y_train)


In [ ]:
TOPetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
TOPetreg.fit(X_train, y_train)

In [ ]:
##  Save these four best models

In [ ]:
## optional save models
#joblib.dump(TOPlinr, ('../result_models/' + 'unharm_whole_top_linr.sav'))
#joblib.dump(TOPllreg, ('../result_models/'+ 'unharm_whole_top_llreg1.sav'))
#joblib.dump(TOPeregr, ('../result_models/'+ 'unharm_whole_top_eregr3.sav'))
#joblib.dump(TOPetreg, ('../result_models/'+ 'unharm_whole_top_etreg4.sav'))

# Running whole TOP model over MRI dataset

In [ ]:
mri_ml_matrix = StrokeMRI.drop('participant_id', axis=1)
X_mri = mri_ml_matrix.drop('age', axis =1)
X_mri = X_mri.values
X_mri = X_mri.astype('float')
y_mri = mri_ml_matrix['age'].values
y_mri= y_mri.astype('float')

In [ ]:
X_mri_test = X_mri
y_mri_test = y_mri


In [ ]:
y_mri_pred = TOPlinr.predict(X_mri_test)

In [ ]:
data= [[
    'linear regression',
    'unharm_whole_top_linr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    TOPlinr.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
linr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#linr_results

In [ ]:
linr_y_test = y_mri_test
linr_y_pred = y_mri_pred
linr_compare = pd.DataFrame(
    {'y_test_real_age': linr_y_test,
     'linr_y_pred_age': linr_y_pred,
    })
linr_compare = linr_compare.reset_index()
linr_compare

In [ ]:
linr_compare.to_csv('whole_top_linr_compare_on_mti.csv')

In [ ]:
y_mri_pred = TOPllreg.predict(X_mri_test)

In [ ]:
data= [[
    'lasso regression',
    'unharm_whole_top_llreg.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    TOPllreg.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
llreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results

In [ ]:
llreg_y_test = y_mri_test
llreg_y_pred = y_mri_pred
llreg_compare = pd.DataFrame(
    {'y_test_real_age': llreg_y_test,
     'linr_y_pred_age': llreg_y_pred,
    })
#llreg_compare = llreg_compare.reset_index()
llreg_compare

In [ ]:
llreg_compare.to_csv('whole_top_llreg_compare_on_mri.csv')

In [ ]:
y_mri_pred = TOPeregr.predict(X_mri_test)

In [ ]:
data= [[
    'elasticnetCV',
    'unharm_whole_top_linr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    TOPeregr.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
eregr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#eregr_results

In [ ]:
eregr_y_test = y_mri_test
eregr_y_pred = y_mri_pred
eregr_compare = pd.DataFrame(
    {'y_test_real_age': eregr_y_test,
     'linr_y_pred_age': eregr_y_pred,
    })
eregr_compare

In [ ]:
eregr_compare.to_csv('whole_top_eregr_compare_on_mri.csv')

In [ ]:
y_mri_pred = TOPetreg.predict(X_mri_test)

In [ ]:
data= [[
    'extra trees',
    'unharm_mri_linr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    TOPetreg.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
etreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#etreg_results

In [ ]:
etreg_y_test = y_mri_test
etreg_y_pred = y_mri_pred
etreg_compare = pd.DataFrame(
    {'y_test_real_age': etreg_y_test,
     'linr_y_pred_age': etreg_y_pred,
    })
#etreg_compare = etreg_compare.reset_index()
etreg_compare

In [ ]:
etreg_compare.to_csv('whole_top_etreg_compare_on_mri.csv')

compile csvs of results

In [ ]:
top_based_unharmonized_on_mri =pd.concat([linr_results,
                   llreg_results,
                   eregr_results,
                  etreg_results],
                  axis=0)
top_based_unharmonized_on_mri

In [ ]:
top_based_unharmonized_on_mri.to_csv('whole_top_based_unharmonized_on_mri.csv')